<a href="https://colab.research.google.com/github/MTia05/Visualizzazione-Scientifica/blob/main/codici/cartinaItalia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import plotly.express as px
import pandas as pd
import os

# Percorso file
# Trova la cartella dove si trova il notebook
base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Unisce i pezzi in modo corretto
file_path = os.path.join(base_path, "dataSet", "datasetCartinaItalia.csv")

def generate_merged_italy_map():
  # Controllo
    if not os.path.exists(file_name):
        print(f"ERRORE: Il file '{file_name}' non è stato trovato.")
        return

    try:
        # Filtraggio solo 2024
        mask = (
            (df['TIME_PERIOD'] == 2024) &
            (df['MEASURE'] == 'THV') &
            (df['DATA_TYPE'] == '14_FUMO_SI')
        )
        df_filtered = df[mask].copy()

        # Caricamento tabella
        df = pd.read_csv(file_name)

        # Pulizia
        df['Territorio'] = df['Territorio'].str.strip().str.replace('"', '', regex=False)
        df['Territorio'] = df['Territorio'].apply(lambda x: x[1:-1] if (isinstance(x, str) and x.startswith("'") and x.endswith("'")) else x)
        df['Osservazione'] = pd.to_numeric(df['Osservazione'], errors='coerce')

        # Normalizzazione nomi per il GeoJSON di OpenPolis altrimenti non riconosce le regioni!!
        mapping_nomi = {
            "Valle d'Aosta / Vallée d'Aoste": "Valle d'Aosta/Vallée d'Aoste",
            "Trentino Alto Adige / Südtirol": "Trentino-Alto Adige/Südtirol",
            "Friuli-Venezia Giulia": "Friuli Venezia Giulia"
        }
        df_filtered['Territorio'] = df_filtered['Territorio'].replace(mapping_nomi)

        # GeoJSON e Plot !!!SERVE PER AVERE I CONFINI FATTI BENE!!!
        geojson_url = "https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_regions.geojson"

        # Rimuove righe senza valore
        df_plot = df_filtered.dropna(subset=['Osservazione']).copy()

        # Valori per il gradiente
        real_min = df_plot['Osservazione'].min()
        real_max = df_plot['Osservazione'].max()

        # Scala di Blu
        custom_scale = px.colors.sequential.Blues

        fig = px.choropleth(
            df_plot,
            geojson=geojson_url,
            locations='Territorio',
            featureidkey='properties.reg_name',
            color='Osservazione',
            hover_name='Territorio',
            color_continuous_scale=custom_scale,
            range_color=[real_min, real_max],
            labels={'Osservazione': 'Fumatori'}
        )

        # Configurazione sull'Italia
        fig.update_geos(
            visible=False,
            fitbounds="locations",
            showcoastlines=True,
            coastlinecolor="#ffffff",
            showcountries=False,
            lakecolor="#ffffff",
            bgcolor="white"
        )

        # Layout
        fig.update_layout(
            title={
                'text': '<span style="color:#2E4053; font-size:24px; font-weight:bold;">Densità Fumatori in Italia</span><br><span style="color:#95A5A6; font-size:16px;">Dati del 2024 • Valori espressi in migliaia</span>',
                'y': 0.95,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top'
            },
            margin={"r":0,"t":100,"l":0,"b":0},
            coloraxis_colorbar=dict(
                title=dict(
                    text="Migliaia di persone<br> &nbsp;",
                    side="top"
                ),
                thickness=18,
                len=0.5,
                x=0.85,
                y=0.5,
                tickvals=[real_min, (real_min + real_max)/2, real_max],
                ticktext=[f"{real_min:.0f}", f"{(real_min + real_max)/2:.0f}", f"{real_max:.0f}"],
                tickmode="array"
            )
        )

        # Hover e bordi
        fig.update_traces(
            marker_line_width=0.5,
            marker_line_color="#444444",
            hovertemplate="<b>%{hovertext}</b><br>Fumatori: %{z:.0f} mila persone<extra></extra>"
        )

        fig.show()

    except Exception as e:
        print(f"Errore durante l'elaborazione: {e}")

if __name__ == "__main__":
    generate_merged_italy_map()